#### Prerequisite

In [1]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


#### Create a docker image

This step can run in SageMaker classic notebook environment or in your own local environment with docker installed.

In [2]:
# Image name below preferably starts with the prefix `sagemaker`
!docker build -t sagemaker_text_encoder -f Dockerfile .

Sending build context to Docker daemon    440MB
Step 1/7 : FROM python:3.7
 ---> 5cda39795abb
Step 2/7 : COPY requirements.txt ./
 ---> Using cache
 ---> 1588774ea606
Step 3/7 : RUN python -m pip install --upgrade pip
 ---> Using cache
 ---> 9785af30ece0
Step 4/7 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Using cache
 ---> 43f90ccb97ea
Step 5/7 : COPY serve /usr/local/bin
 ---> Using cache
 ---> 718d4180f98b
Step 6/7 : RUN chmod +x /usr/local/bin/serve
 ---> Using cache
 ---> cc551675acc3
Step 7/7 : EXPOSE 8080
 ---> Using cache
 ---> d722ce563cc8
Successfully built d722ce563cc8
Successfully tagged sagemaker_text_encoder:latest


#### Push docker image from local to ECR

In [3]:
%%sh

# Specify a name to your custom container
container_name=sagemaker_text_encoder  # should match name from previous cell
echo "Container Name: " ${container_name}

# Retreive AWS account ID
account=$(aws sts get-caller-identity --query Account --output text)

# Get the AWS region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "ECR Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:latest"
echo "ECR Image URI: " ${image}

# If the ECR repository does not exist, create it.
aws ecr describe-repositories --repository-names ${container_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${container_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Tag the local image with ECR image name
docker tag ${container_name} ${image}

# Finally, push the local docker image to ECR with the full ECR image name
docker push ${image}

Container Name:  sagemaker_text_encoder
Account:  119174016168
Region: us-east-1
ECR Repository:  119174016168.dkr.ecr.us-east-1.amazonaws.com
ECR Image URI:  119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker_text_encoder:latest
Login Succeeded
The push refers to repository [119174016168.dkr.ecr.us-east-1.amazonaws.com/sagemaker_text_encoder]
7b4903b5f373: Preparing
d88ab4d0756e: Preparing
e755b640ef85: Preparing
64c5ff71ede4: Preparing
ffc335b17c21: Preparing
eef135e35b6e: Preparing
a0db21004f62: Preparing
0b53caaeb40b: Preparing
1cad4dc57058: Preparing
4ff8844d474a: Preparing
b77487480ddb: Preparing
cd247c0fb37b: Preparing
cfdd5c3bd77e: Preparing
870a241bfebd: Preparing
eef135e35b6e: Waiting
a0db21004f62: Waiting
0b53caaeb40b: Waiting
1cad4dc57058: Waiting
4ff8844d474a: Waiting
b77487480ddb: Waiting
cd247c0fb37b: Waiting
cfdd5c3bd77e: Waiting
870a241bfebd: Waiting
d88ab4d0756e: Layer already exists
e755b640ef85: Layer already exists
ffc335b17c21: Layer already exists
64c5ff71ede

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



##### Uncomment and run the command below if your local images need to be cleaned.

In [4]:
#!docker rmi $(docker images -q) -f

## Deploy feature encoder as a SageMaker real time endpoint

#### Imports 

In [5]:
from time import gmtime, strftime
import sagemaker
import datetime
import boto3
import time

#### Essentials

In [6]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
sagemaker_client = boto3.client('sagemaker', region_name=region)
image_name = 'sagemaker_text_encoder'
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest'
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

#### Pack all encoders into a tar file and push to S3

In [7]:
!tar -czf text-encoder.tar.gz -C ./data .

In [8]:
default_bucket = session.default_bucket()
default_bucket

'sagemaker-us-east-1-119174016168'

In [9]:
!aws s3 cp text-encoder.tar.gz s3://{default_bucket}/

upload: ./text-encoder.tar.gz to s3://sagemaker-us-east-1-119174016168/text-encoder.tar.gz


#### Create a SageMaker model object

In [10]:
model_name = f'text-encoder-{current_timestamp}'
model_artifacts_location = f's3://{default_bucket}/text-encoder.tar.gz'

In [11]:
response = sagemaker_client.create_model(ModelName=model_name, 
                                         Containers=[{
                                             'Image': image_uri, 
                                             'Mode': 'SingleModel', 
                                             'ModelDataUrl': model_artifacts_location}], 
                                         ExecutionRoleArn=role)
response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:119174016168:model/text-encoder-2022-12-16-19-55-41',
 'ResponseMetadata': {'RequestId': '1aa63f98-5597-450b-93a8-868750e23cca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1aa63f98-5597-450b-93a8-868750e23cca',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Fri, 16 Dec 2022 19:56:12 GMT'},
  'RetryAttempts': 0}}

#### Create a SageMaker endpoint configuration

In [12]:
endpoint_config_name = f'text-encoder-{current_timestamp}'

In [13]:
response = sagemaker_client.create_endpoint_config(EndpointConfigName=endpoint_config_name, 
                                                   ProductionVariants=[{
                                                       'VariantName': 'v1', 
                                                       'ModelName': model_name, 
                                                       'InstanceType': 'ml.c5.xlarge', 
                                                       'InitialInstanceCount': 2, 
                                                       'InitialVariantWeight': 1
                                                   }])
response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:119174016168:endpoint-config/text-encoder-2022-12-16-19-55-41',
 'ResponseMetadata': {'RequestId': 'd9e15b8e-47f6-4cca-a5ab-a21405d82016',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd9e15b8e-47f6-4cca-a5ab-a21405d82016',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'date': 'Fri, 16 Dec 2022 19:56:13 GMT'},
  'RetryAttempts': 0}}

#### Create a SageMaker endpoint

In [14]:
endpoint_name = f'text-encoder-{current_timestamp}'

In [15]:
response = sagemaker_client.create_endpoint(EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)
response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:119174016168:endpoint/text-encoder-2022-12-16-19-55-41',
 'ResponseMetadata': {'RequestId': '100d3579-5bdb-4056-83d1-e6ba02483074',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '100d3579-5bdb-4056-83d1-e6ba02483074',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Fri, 16 Dec 2022 19:56:13 GMT'},
  'RetryAttempts': 0}}

#### Describe endpoint to track creation status

In [16]:
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = response['EndpointStatus']

while endpoint_status == 'Creating':
    time.sleep(15)
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = response['EndpointStatus'] 
    print(endpoint_status)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


#### Invoke endpoint to test deployed feature encoder

In [17]:
sagemaker_runtime = boto3.client('sagemaker-runtime', 
                                 region_name=region)

In [18]:
raw_payload = b"I purchased this headphones on Black Friday sale. Poor quality. Not worth the money."

In [19]:
%%time

response = sagemaker_runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                             Body=raw_payload, 
                                             ContentType='text/csv')
response

CPU times: user 11.9 ms, sys: 101 µs, total: 12 ms
Wall time: 1.25 s


{'ResponseMetadata': {'RequestId': '99212f71-15a4-48ee-abfb-131a0ec5aa25',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '99212f71-15a4-48ee-abfb-131a0ec5aa25',
   'x-amzn-invoked-production-variant': 'v1',
   'date': 'Fri, 16 Dec 2022 19:58:31 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '15489'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'v1',
 'Body': <botocore.response.StreamingBody at 0x7f5ac8c17820>}

##### Extract feature vector from the response

In [20]:
feature_vector = response['Body'].read().decode('utf-8').strip()
feature_vector

'-0.5063672065734863,0.02691291831433773,-0.21883559226989746,-0.04903881996870041,-0.45220017433166504,-0.012785189785063267,0.2532523274421692,0.6240672469139099,0.23640933632850647,-0.18753862380981445,0.10757631063461304,-0.4399247467517853,-0.0724121555685997,0.2958056628704071,0.1202806830406189,-0.12032034993171692,0.0006345053552649915,0.30661094188690186,0.12856058776378632,-0.12162799388170242,-0.012813800014555454,-0.16330821812152863,-0.11310204863548279,-0.0003417262341827154,-0.29698044061660767,-0.13436968624591827,0.13270355761051178,-0.12359733879566193,0.14219136536121368,0.13176468014717102,-0.10959090292453766,0.3327617943286896,-0.4305773973464966,-0.3280176520347595,-0.04573799669742584,-0.23699122667312622,0.28261250257492065,0.2805051803588867,0.14983497560024261,-0.08238425105810165,-0.024005969986319542,-0.08355547487735748,0.17082443833351135,0.790075957775116,-0.006176157388836145,-0.7580922245979309,-2.6694915294647217,-0.304144024848938,-0.0783451423048973